In [13]:
import pygame
import random
import math

# ---------------- Config ----------------
WORLD_WIDTH, WORLD_HEIGHT = 1600, 900
DT_S = 0.05
NETWORK_RADIUS = 350.0
SAFE_DISTANCE = 250.0
SENSOR_RANGE = 400.0
ROAD_Y = WORLD_HEIGHT // 2
CAR_LENGTH = 40.0

# Colors
COLOR_BG = (24, 28, 36)
COLOR_ROAD = (50, 55, 65)
COLOR_CAR = (90, 200, 255)
COLOR_AMBULANCE = (255, 255, 255)
COLOR_BLOCK = (180, 180, 180)

# ---------------- Vehicle ----------------
class Vehicle:
    def __init__(self, x, y, speed, desired_speed, is_ambulance=False):
        self.x = x
        self.y = y
        self.speed = speed
        self.desired_speed = desired_speed
        self.base_desired_speed = desired_speed
        self.accel_mps2 = 0.0
        self.is_ambulance = is_ambulance
        self.hazard_ahead = None
        self.target_speed = desired_speed
        self.flash_timer = 0
        self.flash = False
        self.side_offset = 0  # for giving way
        self.lane_y = y

        if not is_ambulance:
            self.generate_car_style()
        else:
            self.body_color = COLOR_AMBULANCE
            self.stripe_type = "ambulance"

    def generate_car_style(self):
        colors = [(255, 50, 50), (50, 100, 255), (255, 220, 0), (50, 200, 50),
                  (255, 100, 200), (150, 50, 200), (255, 140, 0), (0, 200, 200),
                  (240, 240, 240), (40, 40, 40)]
        self.body_color = random.choice(colors)
        self.stripe_type = random.choice([None, "racing", "horizontal", "number"])
        if self.stripe_type == "number":
            self.number = random.randint(1, 99)

    def step(self, dt):
        # Smooth speed adjustment
        delta = self.target_speed - self.speed
        accel_limit = 2.5 if delta > 0 else -6.0
        self.accel_mps2 = max(-6.0, min(2.5, delta / dt))
        self.speed += self.accel_mps2 * dt
        self.speed = max(0, self.speed)

        # Update ambulance flashing
        if self.is_ambulance:
            self.flash_timer += 1
            if self.flash_timer % 15 == 0:
                self.flash = not self.flash

        # Smooth return of y position if moved aside
        if not self.is_ambulance:
            self.y += (self.lane_y + self.side_offset - self.y) * 0.1

        self.x += self.speed * dt
        if self.x > WORLD_WIDTH + 50:
            self.x = -300
            self.speed = min(self.speed, 15.0)

# ---------------- Simulation ----------------
class Simulation:
    def __init__(self, scenario="ambulance"):
        self.scenario = scenario
        self.vehicles = []
        self.time_s = 0.0
        self.create_vehicles()

    def create_vehicles(self):
        lanes = [ROAD_Y - 60, ROAD_Y, ROAD_Y + 60]

        # Create normal cars
        for i in range(15):
            lane = random.choice(lanes)
            x_pos = (i * 150) % WORLD_WIDTH
            v = Vehicle(x=x_pos, y=lane,
                        speed=random.uniform(18, 23),
                        desired_speed=25.0)
            self.vehicles.append(v)

        # Add ambulance
        if self.scenario == "ambulance":
            self.ambulance = Vehicle(x=-200, y=ROAD_Y, speed=25, desired_speed=35, is_ambulance=True)
            self.vehicles.append(self.ambulance)
        else:
            self.ambulance = None

    def step(self):
        self.vehicles.sort(key=lambda v: v.x)

        ambulance = self.ambulance if self.ambulance else None

        for v in self.vehicles:
            # Normal behavior for cars
            v.target_speed = v.base_desired_speed

            if ambulance and not v.is_ambulance:
                # Distance to ambulance behind
                dx = ambulance.x - v.x
                dy = abs(ambulance.y - v.y)
                if -250 < dx < 200 and dy < 80:
                    # Give way smoothly
                    if v.y < ROAD_Y:
                        v.side_offset = -30
                    else:
                        v.side_offset = 30
                    v.target_speed = v.base_desired_speed * 0.8  # slow slightly
                else:
                    v.side_offset *= 0.95  # return slowly

            v.step(DT_S)

        # Move ambulance
        if ambulance:
            ambulance.step(DT_S)

        # Prevent overlap in same lane
        for i, v1 in enumerate(self.vehicles):
            for v2 in self.vehicles[i+1:]:
                if abs(v1.y - v2.y) < 20:
                    dist = abs(v1.x - v2.x)
                    if dist < CAR_LENGTH + 5:
                        overlap = (CAR_LENGTH + 5) - dist
                        if v1.x > v2.x:
                            v1.x += overlap / 2
                            v2.x -= overlap / 2
                        else:
                            v2.x += overlap / 2
                            v1.x -= overlap / 2

        self.time_s += DT_S

# ---------------- Visualization ----------------
class Visual:
    def __init__(self, sim, fps=30):
        pygame.init()
        self.screen = pygame.display.set_mode((WORLD_WIDTH, WORLD_HEIGHT))
        pygame.display.set_caption("Ambulance Scenario")
        self.clock = pygame.time.Clock()
        self.sim = sim
        self.font = pygame.font.SysFont("Arial", 18)
        self.fps = fps

    def draw(self):
        self.screen.fill(COLOR_BG)
        pygame.draw.rect(self.screen, COLOR_ROAD, (0, ROAD_Y - 120, WORLD_WIDTH, 240))

        # Lane dividers
        for y_offset in [-60, 60]:
            for x in range(0, WORLD_WIDTH, 40):
                pygame.draw.line(self.screen, (80, 80, 80),
                                 (x, ROAD_Y + y_offset),
                                 (x + 20, ROAD_Y + y_offset), 2)

        # Draw vehicles
        for v in self.sim.vehicles:
            car_x = int(v.x - CAR_LENGTH / 2)
            car_y = int(v.y - 10)
            car_w = int(CAR_LENGTH)
            car_h = 20

            pygame.draw.rect(self.screen, v.body_color, (car_x, car_y, car_w, car_h))
            pygame.draw.rect(self.screen, (0, 0, 0), (car_x, car_y, car_w, car_h), 2)

            # Ambulance flashing
            if v.is_ambulance and v.flash:
                pygame.draw.rect(self.screen, (255, 0, 0), (car_x + 5, car_y - 5, 10, 5))
                pygame.draw.rect(self.screen, (0, 0, 255), (car_x + car_w - 15, car_y - 5, 10, 5))

        txt = self.font.render(f"Scenario: {self.sim.scenario} | Time: {self.sim.time_s:.1f}s", True, (220, 230, 245))
        self.screen.blit(txt, (10, 10))

    def loop(self, duration_s=60):
        end_t = self.sim.time_s + duration_s
        running = True
        while running and self.sim.time_s < end_t:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            self.sim.step()
            self.draw()
            pygame.display.flip()
            self.clock.tick(self.fps)

        pygame.quit()

# ---------------- Run ----------------
sim = Simulation(scenario="ambulance")
vis = Visual(sim)
vis.loop(duration_s=60)
